<a href="https://colab.research.google.com/github/YewonKan/DEVOIR/blob/master/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [2]:
import json
import torch
import torch.nn as nn
from transformers import Wav2Vec2Model, Wav2Vec2Processor

class SSLModel(nn.Module):
    def __init__(self, device):
        super(SSLModel, self).__init__()
        self.processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
        self.model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
        self.device = device
        self.out_dim = self.model.config.hidden_size

    def extract_feat(self, input_data):
        if input_data.ndim == 3:
            input_data = input_data.squeeze(1)
        input_data = input_data.cpu().numpy()
        inputs = self.processor(input_data, sampling_rate=16000, return_tensors="pt", padding=True)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state
        return embeddings

# Load configuration from JSON file
with open("/sample_data/AASIST.conf", "r") as f:
    config = json.load(f)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Extract model configuration
args = config["model_config"]

# Initialize the model with Hugging Face-based SSLModel
model = Wav2Vec2Model(args, device=device)
model.to(device)
model.eval()
